In [14]:
#change the path to your absolute path, but only to the point of the github repository
# Add your path and comment the others, in that way everyone only needs to comment/uncomment
#all the rest should be exactly the same because I am using relative paths from this point

#%cd /Users/kroman/Documents/SOHAR/Localenvpc11
%cd C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar

C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar


In [85]:
#commented this out for now because it takes a lot of time and is not strictly necessary at this point
# from roboflow import Roboflow
# rf = Roboflow(api_key="1gq5kPILtMw4wF64rMAM")
# project = rf.workspace("oman-sipc").project("fenderdefectdataweek7label")
# dataset = project.version(1).download("yolov8")

In [5]:
#added this snippet so that the predicition code can also be run with just CPU (so on the work laptop 
#in the future without errors appearing)
import torch
from torch.backends import cudnn

# Check if GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device('cuda:0')
    print("CUDA Available: Yes")
    print("Using GPU.")
    print("cuDNN Available: ", cudnn.is_available())
    print("cuDNN version: ", cudnn.version())
    print("Current CUDA Device: ", torch.cuda.current_device())
    print("CUDA Device Count: ", torch.cuda.device_count())
    print("CUDA Device Name: ", torch.cuda.get_device_name(0))
else:
    # Set the device to CPU
    device = torch.device('cpu')
    print("CUDA Available: No")
    print("Using CPU.")

# Your code can now use the 'device' variable to place tensors and models on the chosen device.
# For example:
# tensor_on_device = your_tensor.to(device)
# model_on_device = your_model.to(device)


C:\Users\s2018293031\Toon\Localenv\local03\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA Available: Yes
Using GPU.
cuDNN Available:  True
cuDNN version:  8302
Current CUDA Device:  0
CUDA Device Count:  1
CUDA Device Name:  NVIDIA RTX A2000 12GB


In [7]:
from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n-cls Classify model
# model_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/ultralytics-main/runs/train_old/best.pt"
# modelbest = YOLO(model_path)
# not necessary if in Git repository directory- uncomment the two lines above and comment out two lines below 

model_path = "Localyolo/ultralytics-main/runs/train_old/best.pt"
modelbest = YOLO(model_path)

# Run inference on a video
# video_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Blanco/Side2.mp4"
# result_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Predicted"

video_path = "Localyolo/TestVideos/Blanco/Side2.mp4"

result_path = "Localyolo/TestVideos/Predicted"

# Check if the results_path directory exists
if not os.path.exists(result_path):
    # Create the directory if it doesn't exist
    os.makedirs(result_path)
    print(f"Directory '{result_path}' created.")
else:
    print(f"Directory '{result_path}' already exists.")


Directory 'Localyolo/TestVideos/Predicted' created.


In [8]:
results = modelbest.predict(video_path, save=True, save_crop=True, project=result_path)  # results list for each frame , save_crop=True, save_txt=True
#print(results)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/315) C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar\Localyolo\TestVideos\Blanco\Side2.mp4: 384x640 1 Missing plate, 35.5ms
video 1/1 (2/315) C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar\Localyolo\TestVideos\Blanco\Side2.mp4: 384x640 1 Missing plate, 16.9ms
video 1/1 (3/315) C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar\Localyolo\TestVideos\Blanco\Side2.mp4: 384x640 1 Missing plat

In [15]:
import pickle

# Assuming results_object is the variable containing your Results object
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [24]:
from datetime import timedelta
import cv2
import pandas as pd
from config import *

#this can be used later if we want to centralize video_paths and best_model from dictionaries stored in config.py
# def process_results(video_key, results):
#     video_path = video_paths.get(video_key)
#     if video_path is None:
#         raise ValueError(f"Video key '{video_key}' not found in config.py")

def process_results(video_path, results):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    detection_data = []

    for frame_idx, result in enumerate(results):
        frame_time = frame_idx / fps

        boxes = result.boxes.xyxy

        if result.boxes.conf.numel() > 0:
            confidences = result.boxes.conf.tolist()
        else:
            confidences = None

        if result.boxes.cls.numel() > 0:
            classes = result.boxes.cls.tolist()
        else:
            classes = None

        if confidences is not None and classes is not None and boxes is not None and len(boxes) > 0:
            for box, conf, class_id in zip(boxes, confidences, classes):
                x_min, y_min, x_max, y_max = map(float, box[:4])
                class_name = modelbest.names[int(class_id)]

                detection_data.append({
                    "Timestamp": frame_time,
                    "Class": class_name,
                    "Confidence": conf,
                    "Bounding Box": (x_min, y_min, x_max, y_max)
                })

    df = pd.DataFrame(detection_data)

    df[["Timestamp", "Confidence"]] = df[["Timestamp", "Confidence"]].round(3)
    df["Bounding Box"] = df["Bounding Box"].apply(lambda box: tuple(round(coord, 3) for coord in box))

    df['Frame Number'] = (df['Timestamp'] * fps).astype(int)

    df.insert(0, "Fender ID", 1)
    df.insert(1, "Defect ID", 1)

    defect_id = 1
    last_timestamp = df.iloc[0]["Timestamp"]
    last_class = df.iloc[0]["Class"]

    for index, row in df.iterrows():
        time_difference = row["Timestamp"] - last_timestamp
        same_class = row["Class"] == last_class

        if time_difference > 0.15 or not same_class:
            defect_id += 1
        last_timestamp = row["Timestamp"]
        last_class = row["Class"]

        df.at[index, "Defect ID"] = defect_id

    cap.release()

    return df

df = process_results(video_path, results)

In [25]:
df.head(30)

,Fender ID,Defect ID,Timestamp,Class,Confidence,Bounding Box,Frame Number
0,1,1,0.000,Missing plate,0.331,"(970.711, 153.232, 1009.895, 261.489)",0
1,1,1,0.033,Missing plate,0.308,"(971.479, 155.967, 1008.975, 264.728)",0
2,1,1,0.067,Missing plate,0.305,"(970.735, 156.478, 1009.937, 267.423)",2
3,1,1,0.100,Missing plate,0.330,"(971.417, 160.757, 1010.166, 269.696)",3
4,1,1,0.133,Missing plate,0.323,"(972.075, 163.164, 1011.536, 273.35)",3
5,1,1,0.167,Missing plate,0.252,"(971.197, 166.067, 1012.296, 273.624)",5
6,1,1,0.200,Missing plate,0.292,"(971.518, 166.85, 1014.086, 278.072)",6
7,1,1,0.233,Missing plate,0.328,"(971.874, 167.946, 1016.162, 282.413)",6
8,1,1,0.267,Missing plate,0.348,"(972.615, 171.177, 1016.103, 285.875)",8
9,1,1,0.300,Missing plate,0.368,"(973.225, 171.816, 1016.03, 288.206)",9


In [20]:
print(results[0].save_dir)

Localyolo\TestVideos\Predicted\predict


In [26]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pandas as pd
from PyPDF2 import PdfMerger


# Set up PDF file
path_topdf = results[0].save_dir
pdf_filename = f"{path_topdf}/detection_report.pdf"
pdf_pages = PdfPages(pdf_filename)

# Create a figure and axes for the table
fig, ax = plt.subplots(figsize=(8.5, 11))
ax.axis('off')  # Turn off axis labels and ticks

# Create a table from the DataFrame and add it to the axes
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)  # Increase table size for better readability

# Adjust column widths based on content
table.auto_set_column_width([i for i in range(len(df.columns))])

# Save the current figure to the PDF file
pdf_pages.savefig(fig, bbox_inches='tight')
plt.close()

# Close the PDF file
pdf_pages.close()

# Optionally, you can add a title page to the PDF using reportlab
title_page_filename = f"{path_topdf}/title_page.pdf"
canvas_obj = canvas.Canvas(title_page_filename, pagesize=letter)
canvas_obj.setFont("Helvetica", 14)
canvas_obj.drawCentredString(300, 700, "Detection Report")
canvas_obj.save()

# Create a PdfMerger object
merger = PdfMerger()

# Add the title page and dataframe to the merged PDF
merger.append(title_page_filename)
merger.append(pdf_filename)

# Output the final merged PDF
merged_pdf_filename = f"{path_topdf}/final_report.pdf"
merger.write(merged_pdf_filename)
merger.close()


In [ ]:
print(results[0].boxes)